In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('project').getOrCreate()

In [2]:
data = spark.read.csv('smsspamcollection/SMSSpamCollection', inferSchema=True, sep='\t')

In [4]:
data.show(5)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
+----+--------------------+
only showing top 5 rows



In [5]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [6]:
data.show(5)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
+-----+--------------------+
only showing top 5 rows



In [7]:
from pyspark.sql.functions import length

In [8]:
data = data.withColumn('length', length(data['text']))

In [9]:
data.show(5)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
+-----+--------------------+------+
only showing top 5 rows



In [10]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [11]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [12]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'], outputCol='features')

In [15]:
from pyspark.ml.classification import NaiveBayes

In [16]:
nb = NaiveBayes()

In [17]:
from pyspark.ml import Pipeline

In [18]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])

In [19]:
cleaner = data_prep_pipe.fit(data)

In [20]:
clean_data = cleaner.transform(data)

In [21]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [22]:
clean_data = clean_data.select('label', 'features')

In [24]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [25]:
training, test = clean_data.randomSplit([0.7,0.3])

In [26]:
spam_detector = nb.fit(training)

In [27]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [28]:
test_results = spam_detector.transform(test)

In [29]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,13,...|[-605.82575148782...|[1.0,1.5176678701...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-997.15672412603...|[1.0,2.1639382290...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-874.84344967108...|[1.0,1.5198321364...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-540.13804613326...|[1.0,5.1108633297...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-443.39131345074...|[1.0,1.9289545523...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-693.21828277810...|[1.0,5.0633344128...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-983.17452722904...|[0.99999999999997...|       0.0|
|  0.0|(13424,[0,1,24,31...|[-356.30740595543...|[1.0,6.9157610719...|       0.0|
|  0.0|(13424,[0,1,43,69...|[-631.30634434388...|[2.07678528510247...|       1.0|
|  0.0|(13424,[0

In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [31]:
acc_eval = MulticlassClassificationEvaluator()

In [32]:
acc = acc_eval.evaluate(test_results)

In [33]:
print(f'accuracy:{acc}')

accuracy:0.9262620505411001
